In [1]:
import sys; sys.path.append('..')
from osp import *
pd.options.display.max_colwidth = 200

In [2]:
ids_parsed =list(set(STASH_SLICES_NLP.keys()))
id = ids_parsed[0]

In [13]:
def get_sent_stats(sent, norm=True):
    tree = get_sent_tree(sent)
    data = {}
    data['num_words'] = get_num_words(tree)
    data['height'] = tree.height()
    data['num_words_in_dependent_clauses'] = get_num_words_in_dependent_clauses(tree)
    data['num_words_in_independent_clauses'] = get_num_words_in_independent_clauses(tree)
    data['num_independent_clauses'] = get_num_independent_clauses(tree)
    num_dependent_clauses = get_num_dependent_clauses(tree)

    if norm:
        for k in data:
            if k!='num_words':
                data[k] = data[k] / data['num_words']
    return data


In [18]:
BAD_POS = {'X'}
BAD_DEPREL = {'flat'}
STASH_SLICE_FEATS = HashStash('osp_slices_1000_feats')
STASH_SLICE_FEATS.clear()

def gen_slice_feats(id, context_len=CONTEXT_LEN, force=False, return_dict=False):
    if not force and id in STASH_SLICE_FEATS:
        return STASH_SLICE_FEATS.get(id)
    else:
        docstr = STASH_SLICES_NLP.get(id,{})
        if not docstr: return {}
        
        doc = stanza.Document.from_serialized(docstr)
        if doc is None: return {}

        o = []
        for sent_i,sent in enumerate(doc.sentences):
            for word_i,word in enumerate(sent.words):
                if word.pos in BAD_POS or word.deprel in BAD_DEPREL:
                    continue

                sent_d = get_sent_stats(sent)

                pos = word.xpos
                deprel = word.deprel
                eg_word = word.text.lower()
                eg_context = get_word_context(doc, sent_i, word_i, context_len=context_len).strip()
                
                out_d = {
                    'sent_i': sent_i,
                    'word_i': word_i,
                    'word': eg_word,
                    'pos': pos,
                    'deprel': deprel,
                    'context': eg_context,
                    **{f'sent_{k}':v for k,v in sent_d.items() if k not in {'sent','num_words'}}
                }
                o.append(out_d)
        df1 = pd.DataFrame(o)
        if not return_dict:
            return df1
        
        odx=df1.drop(columns=['sent_i','word_i']).mean(numeric_only=True)
        orig_d=df1.drop(columns=['sent_i','word_i']).mean(numeric_only=True).to_dict()
        pos_d=df1.pos.value_counts().to_dict()
        deprel_d=df1.deprel.value_counts().to_dict()
        odx = {
            **orig_d,
            **{f'pos_{k}':(v/sum(pos_d.values()))*1000 for k,v in pos_d.items()},
            **{f'deprel_{k}':(v/sum(deprel_d.values()))*1000 for k,v in deprel_d.items()}
        }
        STASH_SLICE_FEATS.set(id, odx)
        return odx


In [21]:
def get_slice_feats(id, return_dict=True, force=False):
    return gen_slice_feats(id, return_dict=return_dict, force=force)
    

In [22]:
get_slice_feats(id)

{'sent_height': 0.46490875576151136,
 'sent_num_words_in_dependent_clauses': 0.644516278384332,
 'sent_num_words_in_independent_clauses': 0.3554837216156678,
 'sent_num_independent_clauses': 0.044995342988365106,
 'pos_NN': 161.5828524319868,
 'pos_IN': 140.14839241549876,
 'pos_DT': 113.76751854905194,
 'pos_JJ': 83.26463314097279,
 'pos_RB': 66.77658697444352,
 'pos_,': 48.639736191261335,
 'pos_NNS': 48.639736191261335,
 'pos_VBZ': 38.746908491343774,
 'pos_VB': 34.624896949711456,
 'pos_.': 32.97609233305853,
 'pos_VBN': 28.029678483099755,
 'pos_PRP': 24.7320692497939,
 'pos_CC': 23.083264633140974,
 'pos_VBP': 23.083264633140974,
 'pos_VBG': 19.785655399835118,
 'pos_MD': 14.014839241549877,
 'pos_PRP$': 13.190436933223413,
 'pos_TO': 10.717230008244023,
 'pos_WDT': 10.717230008244023,
 'pos_:': 9.892827699917559,
 'pos_HYPH': 9.068425391591097,
 'pos_NNP': 9.068425391591097,
 'pos_JJS': 4.9464138499587795,
 'pos_WP': 4.122011541632316,
 'pos_POS': 3.2976092333058533,
 'pos_RP': 

{'sent_i': 21.369620253164555,
 'word_i': 16.269198312236288,
 'sent_height': 14.637974683544304,
 'sent_num_independent_clauses': 1.4970464135021098,
 'sent_num_dependent_clauses': 1.3383966244725738,
 'sent_num_words_in_dependent_clauses': 17.525738396624472,
 'sent_num_words_in_independent_clauses': 16.104641350210972,
 'sent_num_punct': 3.329957805907173,
 'sent_num_punct_colon': 0.07848101265822785,
 'sent_num_punct_comma': 2.226160337552743,
 'sent_num_parens': 0.0540084388185654,
 'sent_num_words_adverb': 1.7757575757575759,
 'sent_num_words_verb': 4.578902953586498,
 'sent_num_words_noun': 8.10295358649789,
 'sent_num_words_adjective': 3.3864503816793894,
 'pos_NN': 145.1476793248945,
 'pos_IN': 128.27004219409284,
 'pos_DT': 96.20253164556962,
 'pos_JJ': 83.54430379746836,
 'pos_NNS': 76.79324894514768,
 'pos_,': 60.75949367088607,
 'pos_RB': 43.037974683544306,
 'pos_CC': 41.35021097046414,
 'pos_VB': 34.59915611814346,
 'pos_.': 33.755274261603375,
 'pos_VBZ': 29.53586497890

In [42]:
odx

sent_i                                   21.369620
word_i                                   16.269198
sent_height                              14.637975
sent_num_independent_clauses              1.497046
sent_num_dependent_clauses                1.338397
sent_num_words_in_dependent_clauses      17.525738
sent_num_words_in_independent_clauses    16.104641
sent_num_punct                            3.329958
sent_num_punct_colon                      0.078481
sent_num_punct_comma                      2.226160
sent_num_parens                           0.054008
sent_num_words_adverb                     1.775758
sent_num_words_verb                       4.578903
sent_num_words_noun                       8.102954
sent_num_words_adjective                  3.386450
dtype: float64

In [29]:
for k,v in dfx.deprel.value_counts().items():
    odx[f'deprel_{k}'] = v

In [4]:
val

b'\x80\x04\x95\x00\x00\x01\x00\x00\x00\x00\x00X\xe9\x18\x00\x00That is what fixes what they are noticing or responding to.\nB might also notice A\'s similar responses on those occasions.\nIt is because both creatures\' responses to the environment and to each other are similar in these ways that there is conver gence on a single cause.\nNo creature could do it alone.\nWithout shared responses and common routes of interest there would be nothing determinate for creatures\' sounds or marks to hook on to.\nSo here is a social dimension of the conditions needed for language and meaning.\nThe perception of objects present to us in ostensive learning situations is needed to connect our words and thoughts with what they are about.\nPerception is a causal proc ess.\nBut social interaction between perceivers is needed even to fix a cause that could give determinate meaning to an expression or content to a subse quent thought.\nSo there is a social contribution to the presence of an appropriate 

In [2]:
df_meta = get_corpus_metadata(min_year=1925, max_year=2025)
df_phil = df_meta[df_meta['discipline'] == 'Philosophy']
df_lit = df_meta[df_meta['discipline'] == 'Literature']

ycutoff = 1950
ycutoff2 = 2000

comparisons = [
    (
        ('Philosophy', df_phil.index.tolist()),
        ('Literature', df_lit.index.tolist()),
    ),
    # (
    #     ('Early Philosophy', df_phil.query(f'year<{ycutoff}').index.tolist()),
    #     ('Late Philosophy', df_phil.query(f'{ycutoff2}<=year').index.tolist()),
    # ),
    # (
    #     ('Early Literature', df_lit.query(f'year<{ycutoff}').index.tolist()),
    #     ('Late Literature', df_lit.query(f'{ycutoff2}<=year').index.tolist()),
    # ),
    # (
    #     ('Synthese', df_meta.query('journal == "Synthese" & year>=2000').index.tolist()),
    #     ('PMLA', df_meta.query('journal == "PMLA" & year>=2000').index.tolist()),
    # ),
    # (
    #     ('The Philosophical Review', df_meta.query('journal == "The Philosophical Review" & year>=2000').index.tolist()),
    #     ('The Modern Language Review', df_meta.query('journal == "The Modern Language Review" & year>=2000').index.tolist()),
    # )
]

In [3]:
def run_comparison(comparisons, sample_size=5_000, num_runs=25, **kwargs):
    o_preds,o_feats = [],[]
    for groups in comparisons:
        comparison_str = f'{groups[0][0]} vs {groups[1][0]}'
        group_d = {
            'group1': groups[0][0],
            'group2': groups[1][0],
            'comparison': comparison_str,
        }
        print(f'Classifying {comparison_str}')
        df_preds, df_feats = classify_by_feat_counts(
            groups=groups,
            sample_size=sample_size,
            verbose=False,
            num_runs=num_runs,
            **kwargs
        )
        print('Accuracy:', df_preds.accuracy.median() * 100,'\n')
        o_preds.append(df_preds.assign(**group_d))
        o_feats.append(df_feats.assign(**group_d))
    odf_feats = pd.concat(o_feats) if o_feats else None
    odf_preds = pd.concat(o_preds) if o_preds else None
    return odf_preds, odf_feats




In [22]:
df_preds, df_feats = run_comparison(comparisons, None, 10, feat_n_egs=1)

Classifying Philosophy vs Literature
Philosophy 30449 / Literature 22885


Philosophy 22885 / Literature 22885: 100%|██████████| 10/10 [00:17<00:00,  1.79s/it]

Accuracy: 86.68450950404194 



In [23]:
df_preds

,true_label,pred_label,test_label,confidence,correct,accuracy,support,run,group1,group2,comparison
id,,,,,,,,,,,
lit/2872755__03,Literature,Literature,Literature / Philosophy,0.755647,True,0.866594,45770,0,Philosophy,Literature,Philosophy vs Literature
lit/460552__03,Literature,Literature,Literature / Philosophy,0.613857,True,0.866594,45770,0,Philosophy,Literature,Philosophy vs Literature
lit/2872832__02,Literature,Literature,Literature / Philosophy,0.680384,True,0.866594,45770,0,Philosophy,Literature,Philosophy vs Literature
lit/459241__01,Literature,Literature,Literature / Philosophy,0.767360,True,0.866594,45770,0,Philosophy,Literature,Philosophy vs Literature
lit/509674__01,Literature,Literature,Literature / Philosophy,0.936460,True,0.866594,45770,0,Philosophy,Literature,Philosophy vs Literature
...,...,...,...,...,...,...,...,...,...,...,...
phil/10.2307/2380015__01,Philosophy,Philosophy,Literature / Philosophy,0.606777,True,0.866222,45770,9,Philosophy,Literature,Philosophy vs Literature
phil/10.2307/2380965__06,Philosophy,Philosophy,Literature / Philosophy,0.851040,True,0.866222,45770,9,Philosophy,Literature,Philosophy vs Literature
phil/10.2307/2183813__01,Philosophy,Philosophy,Literature / Philosophy,0.721922,True,0.866222,45770,9,Philosophy,Literature,Philosophy vs Literature


In [24]:
df_feats.iloc[0]

feature                    deprel_mark
weight                       22.838095
group1                      Philosophy
group2                      Literature
comparison    Philosophy vs Literature
Name: 29, dtype: object

In [40]:
dfx=df_feats.query('comparison == "Philosophy vs Literature"')
dfx=dfx.groupby(['feature']).mean(numeric_only=True).reset_index()
dfx=get_df_feats_with_pos_mdw(dfx, comparisons[0], feat_n=10, feat_n_egs=2).set_index('feature')
dfx=dfx[[c for c in dfx if not 'rank' in c]]
# dfx

In [41]:
# dfx['odds_ratio_log_abs'] = np.abs(np.log(dfx['odds_ratio']))
# dfx.groupby('feat_type').mean(numeric_only=True).sort_values('odds_ratio_log_abs',ascending=False)

In [42]:
printm('### Most philosophy-like')
display(dfx.sort_values('odds_ratio',ascending=False).drop(columns=['top2','egs2']).head(10).fillna(''))


### Most philosophy-like

,weight,feat_desc,result_desc,fpk1,fpk2,odds_ratio,sum1,sum2,sig,top1,egs1,feat_type
feature,,,,,,,,,,,,
pos_FW,2.287075,Foreign word,Foreign words are 3.8x more common in Philosophy than Literature.,1.0,0.3,3.8,5259.0,699.0,***,i.e. e.g. etc i.e etc. e.g cf eds. pp. c.,"""modern physics, I.E., classical mechanics"" ""E.G. I expect that I""",pos
pos_NFP,1.000696,Superfluous punctuation,Superfluous punctuations are 3.8x more common in Philosophy than Literature.,0.4,0.1,3.8,1975.0,262.0,***,* { | ~ -> · > ^ - -*,"""G < / >(*), we call physical"" ""Axiom P { ext (S) S G""",pos
deprel_goeswith,0.004582,?,?s are 3.0x more common in Philosophy than Literature.,0.0,0.0,3.0,12.0,2.0,,_ na,"""predicate ... = _ , not the open"" ""M. Sos NA et al""",deprel
pos_SYM,1.030822,Symbol,Symbols are 2.7x more common in Philosophy than Literature.,0.4,0.1,2.7,2168.0,395.0,***,= / + { ̄ \ -> \( => //,"""predicates, £ = { E, F,. Individual"" ""of sentences < / >v that are verifiable""",pos
pos_EX,3.306791,Existential there,Existential theres are 2.0x more common in Philosophy than Literature.,3.0,1.5,2.0,16297.0,4021.0,***,there ihere,"""foreseeable bound, THERE is no reason why"" ""IHERE are innumerable""",pos
deprel_expl,5.942931,Expletive,Expletives are 2.0x more common in Philosophy than Literature.,5.8,3.0,2.0,31671.0,8081.0,***,there it neither both that this its here ihere which,"""in time t, THERE will be some set"" ""IT is not satisfactory""",deprel
deprel_ccomp,9.010703,Clausal complement,Clausal complements are 1.9x more common in Philosophy than Literature.,9.9,5.4,1.9,54486.0,14614.0,***,is have has are true be know do what possible,"""asserts that there IS a particular set"" ""indispensable we HAVE no choice but to""",deprel
deprel_csubj:pass,0.166409,?,?s are 1.8x more common in Philosophy than Literature.,0.1,0.1,1.8,728.0,201.0,***,have has having made give become doing intended used given,"""should already HAVE all future possibilities"" ""physical sciences) HAS a highly - differentiated""",deprel
deprel_csubj,2.028510,Clausal subject,Clausal subjects are 1.8x more common in Philosophy than Literature.,2.5,1.4,1.8,13698.0,3782.0,***,say see have think having make note doing believe do,"""thing simply to SAY that philosophers"" ""important to SEE that in the fist""",deprel


In [43]:

printm('----\n### Most literature-like')
display(dfx.sort_values('odds_ratio',ascending=True).drop(columns=['top1','egs1']).head(10).fillna(''))

----
### Most literature-like

,weight,feat_desc,result_desc,fpk1,fpk2,odds_ratio,sum1,sum2,sig,top2,egs2,feat_type
feature,,,,,,,,,,,,
pos_NNPS,-3.420410,"Proper noun, plural","Proper noun, plurals are 4.0x more common in Literature than Philosophy.",0.6,2.2,0.3,3073.0,6109.0,***,states jews americans ages works studies letters indians christians tales,"""the United STATES of the latter nineteenth"" ""possibilities that the JEWS and their conceptually""",pos
pos_NNP,-38.123874,"Proper noun, singular","Proper noun, singulars are 2.4x more common in Literature than Philosophy.",18.7,44.0,0.4,102299.0,120129.0,***,god shakespeare milton english mr. new john wordsworth england james,"""their duty to GOD, to their mother"" ""astonishment that SHAKESPEARE, let us say""",pos
pos_GW,-0.004643,?,?s are 2.4x more common in Literature than Philosophy.,0.0,0.0,0.4,5.0,6.0,,michael jenny futuri raymond geschichtsdrama herbert,"""MICHAEL PERRAUDIN"" ""JENNY MEZCIEMS""",pos
pos_VBD,-20.022553,"Verb, past tense","Verb, past tenses are 2.7x more common in Literature than Philosophy.",5.7,15.2,0.4,31316.0,41398.0,***,was had were did made wrote came became said took,"""the room there WAS little furniture"" ""These women then HAD to redefine themselves""",pos
deprel_dislocated,-0.086189,?,?s are 2.0x more common in Literature than Philosophy.,0.0,0.1,0.5,196.0,192.0,***,i what whatever one issue those audience love lady ladies,"""into alertness (I. Lazaro s first"" ""I take in, WHAT I dismiss, what""",deprel
deprel_vocative,-0.322118,?,?s are 2.2x more common in Literature than Philosophy.,0.1,0.3,0.5,801.0,883.0,***,i cf sir lord op lady me god friend t,"""I. Of Generic Worlds"" ""CF. He is so unquestioning""",deprel
pos_POS,-9.237521,Possessive ending,Possessive endings are 2.2x more common in Literature than Philosophy.,4.0,8.6,0.5,21822.0,23485.0,***,'s ' s 'n 'd -s mir ofthe uns }s,"""the audience S perception through"" ""the villagers understanding of""",pos
deprel_orphan,-0.021993,?,?s are 1.5x more common in Literature than Philosophy.,0.0,0.0,0.6,148.0,114.0,***,particular time all then instance england end general ourselves hu,"""violence and in PARTICULAR on assassination"" ""for the second TIME signs in blood""",deprel
pos_PRP$,-14.516863,Possessive pronoun,Possessive pronouns are 1.6x more common in Literature than Philosophy.,10.1,15.9,0.6,55369.0,43291.0,***,his its their her my our your thy ye thine,"""perception through HIS disposition of"" ""art can attain ITS fullest effect""",pos


In [20]:
printm('### Not significant')
# display(dfx[dfx.sig!='***'])

### Not significant

In [21]:
dfx.loc[[i for i in dfx.index if 'sent_' in i]].sort_values('weight',ascending=False)

,weight,feat_desc,result_desc,fpk1,fpk2,odds_ratio,sum1,sum2,sig,top1,top2,egs1,egs2,mdw_rank,mdw1_rank,mdw2_rank,feat_type,feat1_rank,feat2_rank,feat_rank
feature,,,,,,,,,,,,,,,,,,,,
sent_avg_num_sents,0.261379,,,,,,,,,,,,,,,,,25,69,45
sent_avg_height,0.141915,,,,,,,,,,,,,,,,,34,60,54
sent_perc_dependent_clauses,0.015552,,,,,,,,,,,,,,,,,48,46,84
sent_perc_independent_clauses,-0.016949,,,,,,,,,,,,,,,,,58,36,82
sent_avg_num_words_per_clause,-0.345656,,,,,,,,,,,,,,,,,76,18,41
sent_avg_num_words_per_sent,-0.382827,,,,,,,,,,,,,,,,,78,16,36
